In [1]:
from simul import benchmark

In [2]:
%timeit benchmark()
#result 12.9 s ± 83.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

16.6 s ± 2.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
import cProfile
pr = cProfile.Profile()
pr.enable()
benchmark()
pr.disable()
pr.print_stats()

         7023 function calls in 14.428 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   14.428   14.428 <ipython-input-3-58d835d77762>:4(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-3-58d835d77762>:5(<module>)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:142(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:207(pre_run_code_hook)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:1258(user_global_ns)
        2    0.000    0.000   14.428    7.214 interactiveshell.py:3230(run_code)
        2    0.000    0.000    0.000    0.000 ipstruct.py:125(__getattr__)
     3000    0.001    0.000    0.001    0.000 random.py:371(uniform)
        1    0.000    0.000   14.428   14.428 simul.py:114(benchmark)
        1    0.001    0.001    0.003    0.003 simul.py:115(<listcomp>)
  

In [4]:
%prun -f benchmark()
#result
'''
         7007 function calls in 11.932 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   11.929   11.929   11.929   11.929 simul.py:15(evolve)
        1    0.002    0.002    0.003    0.003 simul.py:115(<listcomp>)
     3000    0.001    0.000    0.001    0.000 random.py:371(uniform)
     1000    0.000    0.000    0.000    0.000 simul.py:8(__init__)
     3000    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}
        1    0.000    0.000   11.932   11.932 <string>:1(<module>)
        1    0.000    0.000   11.932   11.932 {built-in method builtins.exec}
        1    0.000    0.000   11.932   11.932 simul.py:114(benchmark)
        1    0.000    0.000    0.000    0.000 simul.py:13(__init__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
'''

"\n         7007 function calls in 11.932 seconds\n\n   Ordered by: internal time\n\n   ncalls  tottime  percall  cumtime  percall filename:lineno(function)\n        1   11.929   11.929   11.929   11.929 simul.py:15(evolve)\n        1    0.002    0.002    0.003    0.003 simul.py:115(<listcomp>)\n     3000    0.001    0.000    0.001    0.000 random.py:371(uniform)\n     1000    0.000    0.000    0.000    0.000 simul.py:8(__init__)\n     3000    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}\n        1    0.000    0.000   11.932   11.932 <string>:1(<module>)\n        1    0.000    0.000   11.932   11.932 {built-in method builtins.exec}\n        1    0.000    0.000   11.932   11.932 simul.py:114(benchmark)\n        1    0.000    0.000    0.000    0.000 simul.py:13(__init__)\n        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}\n"

In [5]:
%load_ext line_profiler

In [23]:
from simul import ParticleSimulator,benchmark 
#優化前
%lprun -f ParticleSimulator.evolve benchmark()
#result
r'''
Timer unit: 3.95062e-07 s

Total time: 136.565 s
File: C:\Users\user\Documents\python\machinelearning\optimize\simul.py
Function: evolve at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    15                                               def evolve(self, dt):
    16         1         13.0     13.0      0.0          timestep = 0.00001
    17         1         15.0     15.0      0.0          nsteps = int(dt/timestep)
    18                                                   
    19     10001      55569.0      5.6      0.0          for i in range(nsteps):
    20  10010000   31380132.0      3.1      9.1              for p in self.particles:
    21                                                           #計算方向
    22  10000000   82864277.0      8.3     24.0                  norm = (p.x**2+p.y**2)**0.5
    23  10000000   40156089.0      4.0     11.6                  v_x = -p.y/norm
    24  10000000   34617088.0      3.5     10.0                  v_y = p.x/norm
    25                                                           
    26                                                           #計算位移
    27  10000000   39929310.0      4.0     11.6                  d_x = timestep * p.ang_vel * v_x
    28  10000000   37301958.0      3.7     10.8                  d_y = timestep * p.ang_vel * v_y
    29                                                           
    30  10000000   40280404.0      4.0     11.7                  p.x += d_x
    31  10000000   39093998.0      3.9     11.3                  p.y += d_y
'''

'\nTimer unit: 3.95062e-07 s\n\nTotal time: 136.565 s\nFile: C:\\Users\\user\\Documents\\python\\machinelearning\\optimize\\simul.py\nFunction: evolve at line 15\n\nLine #      Hits         Time  Per Hit   % Time  Line Contents\n==============================================================\n    15                                               def evolve(self, dt):\n    16         1         13.0     13.0      0.0          timestep = 0.00001\n    17         1         15.0     15.0      0.0          nsteps = int(dt/timestep)\n    18                                                   \n    19     10001      55569.0      5.6      0.0          for i in range(nsteps):\n    20  10010000   31380132.0      3.1      9.1              for p in self.particles:\n    21                                                           #計算方向\n    22  10000000   82864277.0      8.3     24.0                  norm = (p.x**2+p.y**2)**0.5\n    23  10000000   40156089.0      4.0     11.6                  v_x = -p.y/

In [22]:
#優化後
from simul_optimize import ParticleSimulator,benchmark
%lprun -f ParticleSimulator.evolve_fast benchmark()
r'''
Timer unit: 3.95062e-07 s

Total time: 63.3958 s
File: C:\Users\user\Documents\python\machinelearning\optimize\simul_optimize.py
Function: evolve_fast at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    15                                               def evolve_fast(self, dt):
    16         1         15.0     15.0      0.0          timestep = 0.00001
    17         1         18.0     18.0      0.0          nsteps = int(dt/timestep)
    18                                                   
    19      1001       4574.0      4.6      0.0          for p in self.particles:
    20      1000       3492.0      3.5      0.0              t_x_ang = timestep* p.ang_vel
    21  10001000   26026360.0      2.6     16.2              for i in range(nsteps):
    22  10000000   58630315.0      5.9     36.5                  norm = (p.x**2+p.y**2)**0.5
    23  10000000   35248966.0      3.5     22.0                  p.x, p.y = (p.x - t_x_ang*p.y/norm,
    24  10000000   40556655.0      4.1     25.3                             p.y + t_x_ang*p.x/norm)
'''

'\nTimer unit: 3.95062e-07 s\n\nTotal time: 63.3958 s\nFile: C:\\Users\\user\\Documents\\python\\machinelearning\\optimize\\simul_optimize.py\nFunction: evolve_fast at line 15\n\nLine #      Hits         Time  Per Hit   % Time  Line Contents\n==============================================================\n    15                                               def evolve_fast(self, dt):\n    16         1         15.0     15.0      0.0          timestep = 0.00001\n    17         1         18.0     18.0      0.0          nsteps = int(dt/timestep)\n    18                                                   \n    19      1001       4574.0      4.6      0.0          for p in self.particles:\n    20      1000       3492.0      3.5      0.0              t_x_ang = timestep* p.ang_vel\n    21  10001000   26026360.0      2.6     16.2              for i in range(nsteps):\n    22  10000000   58630315.0      5.9     36.5                  norm = (p.x**2+p.y**2)**0.5\n    23  10000000   35248966.0      3.